In [1]:
#necessary libraries
import numpy as np
import pandas as pd
from urllib.request import urlopen
import requests
import os
import datetime as dt
import praw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import fbeta_score, make_scorer, plot_confusion_matrix
import nltk
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import remove_stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import fbeta_score, make_scorer, plot_confusion_matrix

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import FeatureUnion, Pipeline

import math

## **Yahoo finance data**

In [2]:
import yfinance as yf

In [14]:
crypto_list = ["BTC-USD"]

##Downloading dauly data for specified crypto tickers 
data = yf.download(crypto_list, start="2018-01-01", end="2022-11-23", interval = "1d")
data.reset_index(inplace=True)
data.head(10)

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000
1,2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192
2,2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160
3,2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744
4,2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072
5,2018-01-06,17462.099609,17712.400391,16764.599609,17527.000000,17527.000000,18314600448
6,2018-01-07,17527.300781,17579.599609,16087.700195,16477.599609,16477.599609,15866000384
7,2018-01-08,16476.199219,16537.900391,14208.200195,15170.099609,15170.099609,18413899776
8,2018-01-09,15123.700195,15497.500000,14424.000000,14595.400391,14595.400391,16659999744
9,2018-01-10,14588.500000,14973.299805,13691.200195,14973.299805,14973.299805,18500800512


In [15]:
#Defining returns
data_s = data[["Date", "Adj Close"]]
data_s["lgret"] = np.log(data_s["Adj Close"]) - np.log(data_s["Adj Close"].shift(1))
data_s = data_s.iloc[1: , :]
data_s.head(10)

C:\Users\Grigorii\AppData\Local\Temp/ipykernel_19936/2413173839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_s["lgret"] = np.log(data_s["Adj Close"]) - np.log(data_s["Adj Close"].shift(1))


,Date,Adj Close,lgret
1,2018-01-02,14982.099609,0.092589
2,2018-01-03,15201.000000,0.014505
3,2018-01-04,15599.200195,0.025858
4,2018-01-05,17429.500000,0.110945
5,2018-01-06,17527.000000,0.005578
6,2018-01-07,16477.599609,-0.061741
7,2018-01-08,15170.099609,-0.082675
8,2018-01-09,14595.400391,-0.038620
9,2018-01-10,14973.299805,0.025562
10,2018-01-11,13405.799805,-0.110581


## Reddit and NLP algos

In [17]:
reddit_data = pd.read_csv('reddit_data_no_outliers.csv', parse_dates=['Date'])
reddit_data

,Subreddit,Title,Post Text,ID,Score,Total Comments,Creation,Stickied,Date
0,defi,Top 2022 predictions for DeFi & crypto,I wanted to poll the community to see what top...,qypxgn,45,46,2021-11-21 07:36:42,False,2021-11-21
1,defi,What are some of the better unknown defi proto...,Here some I know of\n\nWOW swap - Received gra...,qyr1n5,28,66,2021-11-21 08:58:27,False,2021-11-21
2,defi,Curve.fi pool insane yield,"Hi,\n\nI'm a noob to yield farming, so please ...",qyzz65,34,54,2021-11-21 17:36:19,False,2021-11-21
3,defi,How long before AVAX flips SOL and ADA?,I really don't think it will take long for thi...,qz0eih,4,5,2021-11-21 17:56:20,False,2021-11-21
4,defi,How long before AVAX flips SOL and ADA?,I really don't think it will take long for thi...,qz0eih,3,5,2021-11-21 17:56:20,False,2021-11-21
...,...,...,...,...,...,...,...,...,...
39061,defi,Improving censorship resistance on Ethereum,NaN,z29bey,1,0,2022-11-22 23:51:30,False,2022-11-22
39062,CryptoCurrencyTrading,BTC miners start most aggressive selling in ye...,"In the past three weeks, since the first rumor...",z29cow,4,1,2022-11-22 23:53:05,False,2022-11-22
39063,CryptoCurrency,The market is worried about Coinbase’s solvenc...,NaN,z29e5u,12,58,2022-11-22 23:54:52,False,2022-11-22
39064,defi,Curve releases the crvUSD stablecoin white paper,NaN,z29etg,2,0,2022-11-22 23:55:39,False,2022-11-22


### Merging Reddit posts and BTC Data

In [18]:
df_full = reddit_data.merge(data_s, on='Date')
df_full = df_full[["Date", "ID", "Title", "lgret"]]
df_full = df_full.sort_values(by="Date")
df_full.head(5)

,Date,ID,Title,lgret
0,2021-11-21,qypxgn,Top 2022 predictions for DeFi & crypto,-0.016326
1,2021-11-21,qyr1n5,What are some of the better unknown defi proto...,-0.016326
2,2021-11-21,qyzz65,Curve.fi pool insane yield,-0.016326
3,2021-11-21,qz0eih,How long before AVAX flips SOL and ADA?,-0.016326
4,2021-11-21,qz0eih,How long before AVAX flips SOL and ADA?,-0.016326


### Data marking methodogy

Before we move to training data, we should enphasize our **marking technique**. We will divide posts into three categories: *negative*, *neutral* and *positive* with respect to their effect on BTC returns. This division allows us to distinguish between the news that appear to lead to significant returns from the ones that are associated with almost zero returns (It is wrong to consider both 0.05% and 5% as positive returns as 0.05% is more comparable to -0.05\%). Thus, *neutral* group captures posts that have positive or negative returns around zero

Thus, we identify the following thresholds:

**negative (-1)** if return is less than -2%

**neutral (0)** if return is between -2% and 2%

**positive (1)** if return is higher than 2%

In [19]:
conditions = [
    (df_full['lgret'] < np.log(0.98)),
    (df_full['lgret'] > np.log(0.98)) & (df_full['lgret'] <= np.log(1.02)),
    (df_full['lgret'] > np.log(1.02))
    ]

marking = ['negative', 'neutral', 'positive']
values = [-1, 0, 1]
df_full['marking'] = np.select(conditions, marking)
df_full['marking_num'] = np.select(conditions, values)
df_full.head(5)

,Date,ID,Title,lgret,marking,marking_num
0,2021-11-21,qypxgn,Top 2022 predictions for DeFi & crypto,-0.016326,neutral,0
1,2021-11-21,qyr1n5,What are some of the better unknown defi proto...,-0.016326,neutral,0
2,2021-11-21,qyzz65,Curve.fi pool insane yield,-0.016326,neutral,0
3,2021-11-21,qz0eih,How long before AVAX flips SOL and ADA?,-0.016326,neutral,0
4,2021-11-21,qz0eih,How long before AVAX flips SOL and ADA?,-0.016326,neutral,0


In [20]:
#Checking all categories are present
print(df_full['marking'].unique())

['neutral' 'negative' 'positive']


In [21]:
#proportions of categories
m_freq = df_full['marking'].value_counts()
print(m_freq)

neutral     22084
negative     9664
positive     7318
Name: marking, dtype: int64


### TF-IDF prediction via SVM

In [33]:
from datetime import datetime
from tqdm import tqdm

#testing on one month 2022/10/18-2022/11/22
#generating dates
dates_list = sorted(list(set(df_full["Date"])))
days = len(set(df_full["Date"]))
dates=[]
for i in range(len(dates_list)):
    dates.append(datetime.strftime(dates_list[i], "%Y-%m-%d"))

#We divide into train 80% and test 20%
train_end = int(days*0.8)-1 # for consistency with other prediction schemes
test_days = days - train_end

#######################################################################################################################
#######################################################################################################################
tfidf_pred = []

#Expanding window scheme:
for i in tqdm(range(test_days-1)):
    #train data
    X_train = df_full[(df_full["Date"]>= dates[0]) & (df_full["Date"]<= dates[train_end+i])]["Title"]
    y_train = df_full[(df_full["Date"]>= dates[0]) & (df_full["Date"]<= dates[train_end+i])]["marking_num"]
    
    #test data
    X_test = df_full[df_full["Date"]== dates[train_end+i+1]]["Title"]
    y_test = df_full[df_full["Date"]== dates[train_end+i+1]]["marking_num"]
    
    #tf-idf training model
    tfidfVect =TfidfVectorizer(lowercase=True, strip_accents='unicode')
    X_train_vec = tfidfVect.fit_transform(X_train)
    classifier = SVC(kernel='linear')
    classifier.fit(X_train_vec, y_train)
    
    #tf-idf predicting model
    X_test_vec = tfidfVect.transform(X_test)
    y_predicted = classifier.predict(X_test_vec)
    
    tfidf_pred.append([y_predicted, y_test])

tfidf_pred

100%|██████████| 74/74 [3:49:41<00:00, 186.23s/it]  


[[array([ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  1,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  26426    0
  26425    0
  26424    0
  26423    0
  26422    0
          ..
  26399    0
  26398    0
  26397    0
  26402    0
  26396    0
  Name: marking_num, Length: 66, dtype: int32],
 [array([ 0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0, -1,  0,
          0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
          0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0, -1]),
  26483    0
  26482    0
  26481    0
  26480    0
  26477    0
  26478    0
  26476    0
  26475    0
  26474    0
  26484    0
  26479    0
  26485    0
  26493    0
  26487    0
  26488    0
  26489    0
  26490    0
  26491    0
  26492    0
  

In [76]:
#Averaging among posts per each day
from statistics import mean

tfidf_final = []
pred_mean = []
y_mean = []
for i in range(test_days-1):
    pred_mean.append(round(np.average(tfidf_pred[i][0])))
    y_mean.append(round(np.average(tfidf_pred[i][1])))

tfidf_final.append([pred_mean, y_mean])

In [101]:
#F-beta score and MSE
Fbeta = fbeta_score(tfidf_final[0][1], tfidf_final[0][0], beta=0.5, average='micro') #micro as multiclass
mse = sum((np.array(tfidf_final[0][1])-np.array(tfidf_final[0][0]))^2)/test_days
(Fbeta, mse)

(0.7027027027027027, 1.3066666666666666)

## Word2Vec as predicting model (codes and one day test example)

Below you can find some vital fragments of the code and also prediction example for a specific date

In [102]:
#Dividing into training and test data
X = df_full['Title']
#y = df_full['marking_num']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=42, stratify=y)
X_train = df_full[(df_full["Date"]>= '2022-11-15') & (df_full["Date"]<= '2022-11-17')]["Title"]
X_all = df_full[(df_full["Date"]>= '2022-11-15') & (df_full["Date"]<= '2022-11-18')]["Title"]

In [103]:
X_train

36073    One year later and I stand by it: Crypto.com w...
36074                My apologies to the crypto community…
36077                   My ledger does not turn on at all.
36076    How do you pay with Bitcoin if you don't have ...
36078    Theres a pattern of exchanges selling all thei...
                               ...                        
36761    Bitcoin Drops After Genesis Suspends Withdrawa...
36776    Anyone have insight comparing SeedSigner vs Jade?
36762     What types of coins can be staked on Nanoledger?
36760    Bitcoin Cash as LEGAL TENDER IN St Kitts and N...
36755    What do you think about the recent email from ...
Name: Title, Length: 1196, dtype: object

In [34]:
Vec_all = [nltk.word_tokenize(x) for x in X_all]
Vec_train = [nltk.word_tokenize(x) for x in X_train]

#labels for all train posts
labels_train = df_full["lgret"][X_train.index].values

933

Embedding training and test samples

In [35]:
model_all = Word2Vec(Vec_all, min_count=1, vector_size=32)
model_train = Word2Vec(Vec_train, min_count=1, vector_size=32)

### Alternative Idea of sentiment
Instead of looking at each word in all posts separately, we analyze the sentiment of a post as a whole, i.e. we create embeddings for a text instead of creating vectors for all words in this text. We define the post's vector as an average of all words' vectors in this post

In [36]:
#As in standard approach:
posts_list_all = []
posts_list = []

#train
for i in range(len(Vec_train)):
    inside_post=[]
    for word in Vec_train[i]:
        inside_post.append(model_train.wv[word])
    posts_list.append(inside_post)

#all (train + test day) (here we do for all, including test and train, but then the test values will be separated):
for i in range(len(Vec_all)):
    inside_post=[]
    for word in Vec_all[i]:
        inside_post.append(model_all.wv[word])
    posts_list_all.append(inside_post)
    
#one post = one vector

#train
model_train_text_vec = []
for j in range(len(posts_list)): 
    data = np.array(posts_list[j])
    avg_vec = np.average(data, axis=0)
    model_train_text_vec.append(avg_vec)

#all (train + test day)
model_all_text_vec = []
for j in range(len(posts_list_all)): 
    data = np.array(posts_list_all[j])
    avg_vec = np.average(data, axis=0)
    model_all_text_vec.append(avg_vec)

In [104]:
import scipy
from tqdm import tqdm

#Step 1, cosine similarity b/w train and test
def cos_sim(x, y):
    #checking for nan
    if math.isnan(scipy.spatial.distance.cosine(x, y)):
        return 0
    else:
        return 1-scipy.spatial.distance.cosine(x, y)

#Step 2 KNN predictor
def knn_sentiment_text(model_all, model_train, posts_list ,label, N): #label=marking we set above; all_posts=train+test
    sentiment= []
    d_final = []
    
    #vec_all = nltk.word_tokenize(all_posts)
    #vec_train = nltk.word_tokenize(train_posts)

    vec_all = model_all
    vec_train = model_train
    
    #finding most similar N neighbor for each new post ($new$) i
    #tqdm()
    for al in vec_all:
        
        similar = []
        markings = []
        nominator = []
        d3 = []
        
        for train in vec_train:
            if  (al != vec_train).all(): #if obs is from test sample
                similar.append(cos_sim(al, train))
        similar_sort = sorted(similar)
        most_sim = similar[len(similar)-N:len(similar)] #take N closest neighbours
        
        for i in range(len(vec_train)):
            if cos_sim(al, vec_train[i]) in most_sim:
                markings.append(label[i]) #extract initial sentiment of closet neighbour
                d3.append(cos_sim(al, vec_train[i])**3)
                
        for i in range(len(markings)):
            markings[i]= markings[i]*d3[i] #with nominator[i] list is out of range
        
        if sum(d3)==0:    
            sentiment.append(0)
            d_final.append(sum(d3)/N)
        else:
            sentiment.append(sum(markings)/sum(d3))
            d_final.append(sum(d3)/N)
        
    return [sentiment, d_final]

Thus, we obtain sentiment and sureness for the posts of the test day. And then we construct aggregated measures which result into buy/hold/sell recommendation for (test day + 1)

In [38]:
out_text = knn_sentiment_text(model_all_text_vec, model_train_text_vec, posts_list, labels_train, 10)

100%|██████████| 1232/1232 [05:40<00:00,  3.62it/s]


In [39]:
#Sureness

sureness = sum(np.array(out_text[1]))/len(out_text[1])

#Sentiment
sentiment = sum(np.array(out_text[0])*np.array(out_text[1]))/(sum(np.array(out_text[1])))

(sureness, sentiment)

(0.9031418914457252, 0.0010757674002461978)

## Predicting using rolling window
As our algo is quite computationally intensive, we use rolling window in order not to accumulate too much data which extensively increases duration of computations. This approach also solves another problem since some old posts might be outdated and bring bias to sentiment analysis

In [105]:
from datetime import datetime
from tqdm import tqdm

#testing on one month 2022/10/18-2022/11/18
#generating dates
start_date = datetime.strptime('2022-10-17', "%Y-%m-%d")
dates_generated = pd.date_range(start_date, periods=30)
dates = dates_generated.strftime("%Y-%m-%d")

total_days= len(dates)

#######################################################################################################################
#######################################################################################################################
predicted = []

#Rolling window scheme: 2 days rolled over month
for i in tqdm(range(total_days-2)):
    X_train = df_full[(df_full["Date"]>= dates[i]) & (df_full["Date"]<= dates[i+1])]["Title"]
    X_all = df_full[(df_full["Date"]>= dates[i]) & (df_full["Date"]<= dates[i+2])]["Title"]
    
#######################################################################################################################    
    Vec_all = [nltk.word_tokenize(x) for x in X_all]
    Vec_train = [nltk.word_tokenize(x) for x in X_train]

    #labels for all train posts
    labels_train = df_full["lgret"][X_train.index].values    
    
####################################################################################################################### 
    model_all = Word2Vec(Vec_all, min_count=1, vector_size=32)
    model_train = Word2Vec(Vec_train, min_count=1, vector_size=32)
#######################################################################################################################
    posts_list_all = []
    posts_list = []
    #Obtaining vector for each post:
    
    #train
    for j in range(len(Vec_train)):
        inside_post=[]
        for word in Vec_train[j]:
            inside_post.append(model_train.wv[word])
        posts_list.append(inside_post)

    #all (train + test day) (here we do for all, including test and train, but then the test values will be separated):
    for j in range(len(Vec_all)):
        inside_post=[]
        for word in Vec_all[j]:
            inside_post.append(model_all.wv[word])
        posts_list_all.append(inside_post)
    
    #one post = one vector

    #train
    model_train_text_vec = []
    for j in range(len(posts_list)): 
        data = np.array(posts_list[j])
        avg_vec = np.average(data, axis=0)
        model_train_text_vec.append(avg_vec)

    #all (train + test day)
    model_all_text_vec = []
    for j in range(len(posts_list_all)): 
        data = np.array(posts_list_all[j])
        avg_vec = np.average(data, axis=0)
        model_all_text_vec.append(avg_vec)
        
#####################################################################################################################
    model = knn_sentiment_text(model_all_text_vec, model_train_text_vec, posts_list, labels_train, 10)
    
    #Sureness
    sureness = sum(np.array(model[1]))/len(model[1])

    #Sentiment
    sentiment = sum(np.array(model[0])*np.array(model[1]))/(sum(np.array(model[1])))

    out = [sureness, sentiment]
    predicted.append(out)
######################################################################################################################
    
predicted

100%|██████████| 28/28 [28:06<00:00, 60.24s/it] 


[[0.08469115260951457, -0.011127327349329983],
 [0.1599733293659728, -0.010130621895941161],
 [0.12486713571174661, -0.0044926791591297635],
 [0.06506245024638077, 0.006211910653393114],
 [0.06769259395618998, 0.0018613913940530582],
 [0.07015891543345384, 0.01850815095417243],
 [0.042735078686878106, -0.0113813032552681],
 [0.04661724713807538, 0.03805012559986314],
 [0.15767549955835675, 0.03301719435894765],
 [0.09263872022184963, -0.02360798432162172],
 [0.05154730484478276, 0.015142492579418134],
 [0.0628271597884933, 0.010775490653264441],
 [0.07828838921133693, -0.008822978719624766],
 [0.16768664096071584, -0.006799218852899343],
 [0.1249227872191169, -0.0005124320157943169],
 [0.15448589625737918, -0.016030424015855985],
 [0.061849096419808756, 0.0025011164907873256],
 [0.089432832404937, 0.04533199824767212],
 [0.10332830462161051, 0.0063851825197112],
 [0.03678243324320353, -0.015157392153024793],
 [0.06436494718536824, -0.015587859865226115],
 [0.32595070798417086, -0.10542

In [116]:
#Taking out sentiment
pred_sent = []
for i in range(len(predicted)):
    pred_sent.append(predicted[i][1])

#marking sentiment:
conditions = [
    (pred_sent < np.log(0.98)),
    (pred_sent > np.log(0.98)) & (pred_sent <= np.log(1.02)),
    (pred_sent > np.log(1.02))
    ]

values = [-1, 0, 1]
pred_marked = np.select(conditions, values)

len(pred_marked)

28

In [132]:
#Obtaining predicted values
y_test = []
for i in range(total_days-2):
    y = round(np.average(df_full[df_full["Date"]== dates[i+2]]["marking_num"]))
    y_test.append(y)
len(y_test)

28

In [131]:
#F-beta score and MSE
Fbeta_nlp = fbeta_score(y_test, pred_marked, beta=0.5, average='micro') #micro as multiclass
mse_nlp = sum((np.array(y_test)-np.array(pred_marked))^2)/len(pred_marked)
(Fbeta_nlp, mse_nlp)

(0.6071428571428571, 1.1428571428571428)